# Búsqueda de texto completo
Autor: Eric S. Tellez <eric.tellez@infotec.mx> <br/>

Tal vez la tarea más emblemática de la Recuperación de Información es la búsqueda de _texto completo_.
El problema consiste en dado un corpus grande de documentos, preprocesarlo para crear una estructura de búsqueda que permita resolver consultas de manera eficiente. Una consulta es un texto corto que específica lo que se desea encontrar en la colección. En particular, es un ejemplo de lo que se desea. Esto lleva a que la estructura de búsqueda resuelve búsquedas por similitud.

La similitud, es entonces un tema central, pero para medirla lo primero es tener una representación de los datos que capture las propiedades deseadas (que serán después evaluadas para medir la similitud). La manera más tradicional de hacerlo, es el uso de un modelo basado en bolsa de palabras (BOW). En dicho modelo, el texto es preprocesado, toquenizado y vectorizado.

- El preprocesamiento incluye tratamientos tan simples como eliminar símbolos no deseados, eliminación de variantes léxicas, reducción a raíces o lemas, corrección de ortografía, eliminiación de palabras comunes (stop words). 
- El toquenizado es el proceso donde el texto es partido, en frases u oraciones, y finalmente en palabras y símbolos que son unidades completas. En este punto también es posible realizar normalizaciones, así como también realizar limpieza basada en estadísticas de los términos.
- El vectorizado utiliza el vocabulario de una colección $\{t_i\}$ para generar una matriz de la colección, i.e., un vector por documento.

Al proceso de modelar una colección mediante un vocabulario y luego ser capaces de generar una representación manejable por una computadora se le llama _modelo de lenguaje_.

## Problema de búsqueda
Una vez que se tiene el modelo de lenguaje y que fue usado para vectorizar una colección $X$, la idea es ser capaces de resolver consultas $q \in Q$, i.e., encontrar un subconjunto de $X$ de tamaño $k$ que más se _parezca_ a $q$. Las consultas deben ser codificadas al mismo espacio que los documentos, i.e., espacio vectorial. Entonces el problema se transforma en encontrar los elementos más parecidos, que dada la representación, es conveniente usar el coseno entre vectores:

$$ \cos(u, q) = \frac{ \sum_i {u_i \cdot q_i}}{\sqrt{\sum_i u_i^2} \cdot \sqrt{\sum_i q_i^2}} $$

Así mismo, $d(u, q) = \arccos(\cos(u, q))$ sería el ángulo entre ambos vectores, que además es una métrica. El problema entonces se transforma en encontrar los vecinos más cercanos en la colección, esto es, si deseamos $k$ resultados de una consulta, estaríamos deseando encontrar aquel subconjunto $knn$ de la colección tal que $\sum_{v \in knn} d(v, q)$ sea mínimo comparado con todo subconjunto de tamaño $k$ de la colección de documentos.

## Velocidad de consultas
Para mejorar la solución de consultas, es posible crear una estructura de datos que simplifique el proceso de encontrar el subconjunto $knn$. En este problema, con una representación basada en bolsa de palabras, la estructura más adecuada es el _índice invertido_.


# Índice invertido

Un índice invertido es una representación dispersa de la matriz $W_{m,n}$ formada por $m$ componentes y $n$ documentos, i.e., cada celda $w_{t,i}$ es el peso asignado para el término $t$ que ocurre en el documento $i$. Por construcción, esta matriz tiene una gran cantidad de ceros, por lo que $W$ es altamente dispersa (pocos términos ocurren en un documento).

$$ W \left\{
\begin{array}{rrrr r}
                & \vec x_1& \vec x_2&       & \vec x_n \\
t_1 \rightarrow & w_{1,1} & w_{1,2} & \dots & w_{1,n} \\
t_2 \rightarrow & w_{2,1} & w_{2,2} &       & w_{2,n} \\
                & \vdots  &         & \ddots&         \\
t_m \rightarrow & w_{m,1} & w_{m,2} &       & w_{m,n} \\
\end{array}
\right.
$$

La representación es entonces por fila, a manera de lista de adjacencia; esto es, cada fila $t$ es representada por las tuplas $(i, w_{t,i})$, esto es, un índice invertido es la siguiente estructura $W^*$

$$ W^* \left\{
\begin{array}{rrr}
t_1 & \rightarrow & \{(i, w_{1, i})\} \\
t_2 & \rightarrow & \{(i, w_{2, i})\} \\
\vdots & \vdots   &  \hfill \vdots \hfill \\
t_m & \rightarrow & \{(i, w_{m, i})\} \\
\end{array}
\right.
$$

la tupla es usada siempre y cuando $w > 0$. Las tuplas suelen ordenarse por su identificador de columna, pero también puede usarse el peso según convenga. A las filas suele llamarseles _listas de posteo_ (_posting lists_). Los requerimientos de una matriz densa son altísimos ya que las representaciones de texto suelen ser de muy alta dimensión. Si la representación contiene muchos ceros, como es el caso de una representación basada en un modelo léxico, es posible representar las matrices de manera dispersa y reducir enormemente los requisitos de la memoría. Como se verá a continuación la representación también influye enormemente en los tiempos de procesamiento.

### Búsqueda mediante un índice invertido

La solución na\"ive de una obtener los $k$ documentos más similares es evaluar todos los vectores $\vec{x}_i$, i.e., columnas de $W$, y determinar aquellos más similares, i.e., minimizar $d(\vec{x}_i, q)$.

El índice invertido $W^*$ contiene la información necesaria para realizar esta operación de manera eficiente. Primeramente, es necesario analizar la expresión de $\cos$. El denominador $\sqrt{\sum_i u_i^2} \cdot \sqrt{\sum_i q_i^2}$, en sus partes es estático para cada vector, por lo que se puede preprocesar y no calcular de manera explícita para cada evaluación de $\cos$. Con respecto al numerador corresponde al producto punto entre $\vec{u}$ y $\vec{q}$, $\sum_i u_i \cdot q_i$. Dicho esto, solo es necesario calcular los productos diferentes de cero; así pues, la evaluación eficiente de $\cos$ corresponde con una evaluación eficiente de la intersección de las componentes diferentes de cero. Los algoritmos como SvS, BY o BK, pueden ser de gran ayuda para este cálculo. Note que aunque que los pesos con valor cero no se representan en $W^*$, dicho índice representa información por fila, lo cual no permite hacer operaciones eficientes entre $q$ y los vectores columna $\vec x$ individuales.


Afortunadamente, la evaluación se puede hacer eficiente para todo el conjunto de posibles candidatos (aquellos donde el producto punto contra $q$ sea diferente de cero). Para esto, se toman las componentes diferentes de cero en $q$, se toman las listas de adyacencia de $W^*$ y se procede a unirlas de manera eficiente. El conjunto de identificadores de documento resultado de esta unión será aquel que debe ser evaluado para obtener el conjunto de documentos similares. 
Si uno toma la intersección, que puede ser más veloz de calcular, entonces podrían perderse documentos relevantes; es posible también mandar el problema a un punto intermedio, es decir al problema de $t$-thresholds, donde se recupera un conjunto donde cada uno de los miembros aparece en al menos $t$ listas.
La manera más eficiente, sin embargo, es realizar optimizaciones por filtrado de pesos o mejorando los esquemas de pesado; la idea general entonces es desaparecer entradas de $W*$ de tal forma que la unión sea siempre pequeña. La adecuada optimización de un índice invertido puede hacerlo escalable a niveles realmente impresionantes.

Los algoritmos de BK pueden ser utilizados para calcular la unión y t-thresholds, así como los algoritmos de mezcla clásicos (_merge_). Es posible unir la operación de unión con la operación de producto punto por vector usando los algoritmos adecuados.

- <https://github.com/sadit/InvertedFiles.jl/blob/main/src/invfilesearch.jl>
- <https://github.com/sadit/InvertedFiles.jl/blob/main/src/winvfilesearch.jl>
- <https://github.com/sadit/Intersections.jl/blob/main/src/merge.jl>

# Medidas de calidad (scores)
La medición de la calidad en un sistema de búsqueda es fundamental para obtener un sistema de RI adecuado. La idea básica es que un algoritmo recupere la información adecuada para solventar los requerimientos de las consultas hechas por usuarios. Dicho de otra forma, si se piden $k$ documentos relacionados a una consulta $q$, se medirá que porcentaje de esos $k$ son relevantes para el usuario.
La evaluación de relevancia de un documento es hecha previamente por _usuarios expertos_ en el dominio del corpus y las consultas. A esta función de relevancia se le llama $\textsf{recall}$.

$$ \textsf{recall}(\text{doc. recuperados}, \text{doc. esperados}) = \frac{\left| \text{doc. recuperados} \cap \text{doc esperados} \right|}{\left|\text{doc. esperados}\right|} $$

Note que no se espera precisamente que cada conjunto de resultados sea de tamaño idéntico, aunque esta será la norma en nuestro curso. Para obtener una estadística fiable, la relevancia será promediada para obtener la calidad del modelo o algoritmo ante un conjunto de consultas. Llamaremos $\textsf{macrorecall}$ al promedio de los recalls varias consultas.

$$ \textsf{macrorecall}(R, G) = \frac{1}{|G|} \sum_i \textsf{recall}(R_i, G_i)$$

El conjunto $R$ es el conjunto de resultados recuperados para un conjunto de consultas, mientras que $G$ es un conjunto especial de resultados que suele llamarse _gold standard_, que sería esa el conjunto de resultados fiables obtenidos a través de la evaluación de expertos humanos.

Es costoso y tardado construir un _gold standard_ para una tarea de recuperación de información, y más aún, para conjuntos de datos grandes. Es por eso que en este curso, evaluaremos la bondad de los modelos usados para la representación de los datos y las optimizaciones para búsqueda mediante el uso de **tareas de clasificación**. En ese sentido la relación entre velocidad y calidad que se muestran no deberán tomarse más que de manera informativa ya que no puede ligarse a un sistema de búsqueda en grandes colecciones de documentos.

## Midiendo la calidad usando clasificación
En términos de clasificación se usará un modelo basado en vecinos cercanos ya que son naturalmente implementados con las máquinas de búsqueda. Pero se aconseja el uso de otros clasificadores en tareas de clasificación.

A lo largo del resto del curso, siempre que sea fácil se utilizarán particiones separadas entre entramiento y prueba. Recuerde que no es nuestro objetivo la clasificación efectiva, y nuestro objetivo es medir la diferencia entre modelos de búsqueda y medir la bondad o perdida de alguna característica modificada a un modelo o algoritmo.

### Clasificación de texto
La clasificación de texto es una tarea de recuperación de información que será solo tocada ligeramente en este curso, ya que esta fuera del alcance del mismo. Sin embargo, se usará en una de sus formas más sencillas para evaluación de la calidad de los modelos, como ya se menciono. El problema de clasificación consiste en aprender una función $\phi(texto) \rightarrow etiqueta$ apartir de un conjunto de un corpus etiquetado tal que para cada texto $t_i$ en el corpus, existe una $y_i$ que es la etiqueta de $t_i$, de tal forma que al usar un texto nunca antes visto $\phi(texto')$ la función sea capaz de calcular una etiqueta. La función $\phi$ basada en recuperar los $k$ vecinos cercanos ($knn$) sobre el corpus de entrenamiento y resolver con la etiqueta más popular entre los $knn$.

Hay diferentes tipos de _scores_ que se utilizan, entre ellos el _accuracy_, _precision_, y _recall_, que es un símil de nuestra función de recall. También se suele tener en cuenta combinaciones de estos como _score_ $F_1$, que es la media armónica entre _precision_ y _recall_. Las funciones de _accuracy_, _precision_ y _recall_ se definen con respecto al número de clases (diferentes etiquetas consideradas) y las funciones básicas TP (true positives), TN (true negatives), FP (false positives), FN (false negatives) <https://en.wikipedia.org/wiki/Precision_and_recall>. Dichas funciones son bien conocidas y usaremos las funciones implementadas en paquetes dedicados enfocando en nuestro objetivo de uso de la búsqueda para medir la calidad de un modelo.


## Modelos que toman en cuenta a los usuarios de manera personalizada
Los requerimientos entre usuarios podrían asumirse diferentes, en ese caso, el problema cambiará su forma precisa ya que ahora se deberá cumplir que el resultado de una consulta sea relevante para un usuario específico, que deberá ser modelado de alguna manera para poder generalizarlo. Esta consideración esta fuera del alcance de este curso.

# Ejemplo

El siguiente es un ejemplo de como cambia el desempeño usando una evaluación exhaustiva, un índice invertido y una pequeña optimización basada en modificación de pesos. 

In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, KNearestCenters, TextSearch, StatsBase, LinearAlgebra, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
Threads.nthreads()

64

### Función de clasificación para medir calidad de un modelo

In [3]:
include("knn.jl")

scores (generic function with 1 method)

## Funciones para leer y modelar el corpus

In [4]:
function text_model_and_vectors(
        corpus;
        textconfig=TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[]),
        voc=Vocabulary(textconfig, corpus),
        model=VectorModel(IdfWeighting(), TfWeighting(), voc)
    )
    
    vectors = vectorize_corpus(model, corpus)
    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [5]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_emojispace()
T = text_model_and_vectors(D.text)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})

# Creando los diferentes métodos de búsqueda

In [6]:
# índice invertido
invfile = WeightedInvertedFile(length(T.model.voc))
append_items!(invfile, VectorDatabase(T.vectors))

## indice invertido con filtros al vocabulario
fmodel = filter_tokens(T.model) do t
    7 <= t.ndocs <= 1000  # filtrando los tokens que ocurren en entre 7 y 1000 documentos
end

## usando el modelo reducido de tokens
fT = text_model_and_vectors(D.text, textconfig=T.textconfig, model=fmodel)

finvfile = WeightedInvertedFile(length(fT.model.voc))
append_items!(finvfile, VectorDatabase(fT.vectors))

{WeightedInvertedFile{Nothing, SimilaritySearch.AdjacencyLists.AdjacencyList{IdWeight}} vocsize=4954, n=45000}

In [7]:
display(@htl "The token filters can operate on any valid field of voc (passed as the named tuple <em>t</em>)")

dump(typeof(T.model.voc))

The token filters can operate on any valid field of voc (passed as the named tuple <em>t</em>)

Vocabulary{TokenLookup} <: Any
  lookup::TokenLookup
  textconfig::TextConfig
  token::Vector{String}
  occs::Vector{Int32}
  ndocs::Vector{Int32}
  token2id::Dict{String, UInt32}
  corpuslen::Int64


In [8]:
# evaluación exhaustiva
brute = ExhaustiveSearch(; db=T.vectors, dist=NormalizedCosineDistance())
nothing

In [9]:
Qvectors = VectorDatabase(vectorize_corpus(T.model, Q.text))
fQvectors = VectorDatabase(vectorize_corpus(fT.model, Q.text))
nothing

## La siguiente celda se debe correr 2 veces para remover el costo de compilación

In [10]:
GC.enable(false)
k = 31
@time Iknns, _ = searchbatch(invfile, Qvectors, k)
@time Bknns, _ = searchbatch(brute, Qvectors, k)
@time fIknns, _ = searchbatch(finvfile, fQvectors, k)
GC.enable(true)

  2.317168 seconds (1.38 M allocations: 96.354 MiB, 1727.62% compilation time)
  4.999678 seconds (268.44 k allocations: 18.960 MiB, 5.08% compilation time)
  0.030865 seconds (10.13 k allocations: 1.804 MiB)


false

In [11]:
Threads.nthreads()

64

# El efecto en la calidad

In [13]:
# La diferencia de costos puede ser importante
# por lo que es también necesario saber el impacto con respecto a la calidad perdida
# Recuerde que esta es una evaluación estocástica
let 
    @info scores(Q.labels, knn(Bknns, D.labels))
    @info scores(Q.labels, knn(Iknns, D.labels))
    @info scores(Q.labels, knn(fIknns, D.labels))
end

[ Info: (macrof1 = 0.06309231433007588, macrorecall = 0.06881181054086415, accuracy = 0.07)
[ Info: (macrof1 = 0.06309231433007588, macrorecall = 0.06881181054086415, accuracy = 0.07)
[ Info: (macrof1 = 0.05999268794791886, macrorecall = 0.06398801060401818, accuracy = 0.0648)


# Sobre los resultados
Lo primero que puede observarse en los resultados es que no son tan buenos como podría desearse; recuerde que nuestro objetivo esta en la búsqueda. También se debe tener en cuenta que esta lejos de una solución aleatoria, ya que son 64 etiquetas. Se puede ver como se multiplica la velocidad varias veces con una calidad muy semejante. En lo posterior, veremos como estos números variarán usando diferentes técnicas.

Finalmente, es necesario remarcar que búsqueda el objetivo es dar a los usuarios información de útilidad, y esto suele evaluarse con _gold standards_ generados por humanos. La cantidad de información será mucho mayor que en tareas de clasificación, y muchas veces, la velocidad es primordial, y si los usuarios encuentran de utilidad los resultados se puede decir que el modelo es efectivo.


# Actividades

- ¿Qué es el recall?
- ¿Qué es macro-recall?
- Explique el porque de las mejoras en tiempo. Use análisis de algoritmos para este ejercicio.
- Implemente un índice invertido que sea capaz de mejorar los tiempos de búsqueda tal y como se muestra en este ejercicio. Para el procesamiento del texto y toquenizado use librerias/paquetes externos.
- Reporte mediante un notebook de Jupyter su implementación, use uno de los conjuntos de datos para ejemplificar su uso y medir los desempeños.

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.